In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans Mono'

In [ ]:
from scipy.interpolate import interp1d
from sklearn.preprocessing import LabelEncoder
import itertools

class Preprocessor():
    def __init__(self, gap_max=1.0, seg_dur=5.0, ol_rate=0.5, resamp_gap=0.01):
        self.gap_max = gap_max
        self.seg_dur = seg_dur
        self.ol_rate = ol_rate
        self.resamp_gap = resamp_gap

    def transform(self, df):
        gap_max = self.gap_max
        seg_dur = self.seg_dur
        ol_rate = self.ol_rate
        resamp_gap = self.resamp_gap

        users = sorted(df['user'].unique())
        activities = sorted(df['activity'].unique())

        segments = []
        seg_usrs = []
        seg_acts = []

        for user, activity in itertools.product(users, activities):
            mask_ua = (df['user'] == user) & (df['activity'] == activity)
            if not mask_ua.any():
                continue
            cpnt_nums = np.cumsum( (df.loc[mask_ua, 'timestamp'].diff() > gap_max) | (df.loc[mask_ua, 'timestamp'].diff() < 0) )
            for num in range(cpnt_nums.min(), cpnt_nums.max()+1):
                cpnt = df[mask_ua][cpnt_nums == num]
                cpnt['timestamp'] -= cpnt['timestamp'].min()
                if cpnt['timestamp'].max() < seg_dur:
                    continue
                f = interp1d(cpnt['timestamp'], cpnt[['ax', 'ay', 'az']], axis=0, kind='cubic')
                for i in np.arange( (cpnt['timestamp'].max()-seg_dur)/(seg_dur*(1-ol_rate)), dtype=np.int32 ):
                    t = np.arange(0, seg_dur, resamp_gap) + i*seg_dur*(1-ol_rate)
                    segments.append( f(t) ), seg_usrs.append( user ), seg_acts.append( activity )
        segments = np.array(segments); seg_usrs = np.array(seg_usrs)

        X = segments
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(seg_acts)

        self.seg_usrs = seg_usrs
        self.label_encoder = label_encoder
        return X, y

df = pd.read_csv(
    '../input/WISDM_ar_v1.1/WISDM_ar_v1.1_raw_modified.txt', 
    names=['user', 'activity', 'timestamp', 'ax', 'ay', 'az'],
    header=None)
df['timestamp'] *= 1e-9
df = df[df['timestamp'] != 0]
df = df[~df['timestamp'].duplicated()]
df = df.dropna()
df = df.reset_index(drop=True)

users = sorted(df['user'].unique())
activities = sorted(df['activity'].unique())

In [ ]:
from tools import Embedder, SineFilter

key = (5, 4, 0, 5, 'identity')
seg_dur, lag, reduce, dim_raw, method = key
preprocessor = Preprocessor(seg_dur=seg_dur)
X, y = preprocessor.transform(df)
embedder = Embedder(lag=lag, reduce=reduce, dim_raw=dim_raw, channel_last=True)
pts = embedder.transform(X)
# weights = np.load(f'./output/w_WISDM/{key}_20221110.npy')
# # weights = np.ones(pts.shape[:-1] + (1, ))
# sine_filter = SineFilter(dim=pts.shape[-1], n_filters=16, random_state=42)
# scale = np.average(np.linalg.norm(pts, axis=-1, keepdims=True), axis=-2, weights=weights/np.sum(weights, axis=-2, keepdims=True))[...,np.newaxis]
# fs = sine_filter.apply(pts/scale, weights)
fs = np.empty_like(pts)[..., :1]

In [ ]:
import tensorflow as tf
from tensorflow import keras
import random

class MyModel(keras.Model):
    def __init__(self, n_classes, use_weights=False):
        super(MyModel, self).__init__()
        self.n_classes = n_classes

        self.main_layers = [
            # keras.layers.GRU(
            #     units=512,
            #     # dropout=0.3,
            #     # recurrent_dropout=0.3,
            #     return_sequences=True
            # ),
            keras.layers.GRU(
                units=256,
                # dropout=0.3,
                # recurrent_dropout=0.3,
                return_sequences=True
            ), 
            keras.layers.GRU(
                units=128,
                # dropout=0.3,
                # recurrent_dropout=0.3,
                return_sequences=True
            ),

            keras.layers.Conv1D(
                filters=32,
                kernel_size=16,
                activation='elu'
            ),
            keras.layers.AveragePooling1D(
                pool_size=2
            ),
            keras.layers.Conv1D(
                filters=32,
                kernel_size=16,
                activation='elu'
            ),
            keras.layers.AveragePooling1D(
                pool_size=2
            ),
            keras.layers.Conv1D(
                filters=32,
                kernel_size=16,
                activation='elu'
            ),
            keras.layers.AveragePooling1D(
                pool_size=2
            ),
            # keras.layers.Conv1D(
            #     filters=32,
            #     kernel_size=16,
            #     activation='elu'
            # ),
            # keras.layers.AveragePooling1D(
            #     pool_size=2
            # ),
            # keras.layers.Conv1D(
            #     filters=32,
            #     kernel_size=16,
            #     activation='elu'
            # ),
            # keras.layers.AveragePooling1D(
            #     pool_size=2
            # ),

            keras.layers.Flatten(),
            # keras.layers.Dense(
            #     units=32,
            #     activation='elu'
            # ),
            keras.layers.Dense(
                units=16,
                activation='elu'
            ),
        ]
        self.aux_layers = [
            keras.layers.Flatten(),
            keras.layers.Dense(
                units=16,
                activation='elu'
            )
        ]
        self.final_layers = [
            keras.layers.Dense(
                units=n_classes,
                activation='softmax'
            )
        ]
        self.use_weights = use_weights
    
    def call(self, inputs, training=None):
        outputs, weights = inputs
        for layer in self.main_layers:
            outputs = layer(outputs, training=training)
        for layer in self.aux_layers:
            weights = layer(weights, training=training)
        if self.use_weights:
            outputs = tf.concat([outputs, weights], axis=-1)
        for layer in self.final_layers:
            outputs = layer(outputs, training=training)
        return outputs

rng = np.random.default_rng(42)
usrs_test = rng.choice(users, 12, replace=False)
usrs_val = usrs_test[:6]
usrs_test = usrs_test[6:]
mask_test = np.isin(preprocessor.seg_usrs, usrs_test)
mask_val = np.isin(preprocessor.seg_usrs, usrs_val)
mask_train = ~(mask_val | mask_test)

callbacks = [
    keras.callbacks.EarlyStopping(
        patience=20, 
        min_delta=1e-4,
        restore_best_weights=True
    )
]

keras.backend.clear_session()
for use_weights in (False, True):
    tf.random.set_seed(42); random.seed(42); np.random.seed(42)

    model = MyModel(n_classes=6, use_weights=use_weights)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=keras.optimizers.Adam(1e-3),
        metrics=['accuracy']
    )
    model.fit(
        (X[mask_train], fs[mask_train]), 
        y[mask_train],
        epochs=50,
        batch_size=1<<10,
        callbacks=callbacks,
        validation_data=((X[mask_val], fs[mask_val]), y[mask_val]),
    )
    model.evaluate((X[mask_test], fs[mask_test]), y[mask_test])